In [62]:
import autogen
from autogen import ConversableAgent

In [63]:
config_list = autogen.config_list_from_json(
    env_or_file="AOAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4o"
        }
    },
)

In [64]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    code_execution_config={
        "use_docker": False,
    }
)

In [65]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config= { "config_list": config_list },
    system_message="""
    You are the Engineer. Given a topic, write code to retrieve related papers from the arXiv API, 
    print their title, authors, abstract, and link.
    You write python/shell code to solve tasks. Wrap the code in a code block that specifies 
    the script type. The user can't modify your code. So do not suggest incomplete code which 
    requires others to modify. Don't use a code block if it's not intended to be executed by 
    the executor. Don't include multiple code blocks in one response. 
    Do not ask others to copy and paste the result. Check the execution result returned by 
    the executor. If the result indicates there is an error, fix the error and output the code again. 
    Suggest the full code instead of partial code or code changes. 
    If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
    analyze the problem, revisit your assumption, collect additional info you need, and think of a 
    different approach to try.
    """,
)

In [66]:
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the Coder and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [67]:
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config={ "config_list": config_list },
    system_message="""
    You are the Scientist. Please categorize papers after seeing their abstracts printed 
    and create a markdown table with Domain, Title, Authors, Summary and Link
    """
)

Below is the function to select the speaker deterministically as per need 

In [68]:
import json
from autogen import Agent, GroupChat

def state_transition(last_speaker: Agent, groupchat: GroupChat):
    messages = groupchat.messages
    print(f"Message: {json.dumps(messages)}")
    
    if last_speaker is user_proxy:
        # init -> retrieve
        return engineer
    elif last_speaker is engineer:
        # retrieve: action 1 -> action 2
        return executor
    elif last_speaker is executor:
        if "exitcode: 1" in messages[-1]["content"]:
            # retrieve --(execution failed)--> retrieve
            return engineer
        else:
            # retrieve --(execution success)--> research
            return scientist
    elif last_speaker == "Scientist":
        # research -> end
        return None

In [69]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)

In [70]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    llm_config={ "config_list": config_list }
)

In [71]:
user_proxy.initiate_chat(
    manager, 
    message="Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains."
)


Admin (to chat_manager):

Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.

--------------------------------------------------------------------------------
Message: [{"content": "Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.", "role": "user", "name": "Admin"}]

Next speaker: Engineer

Engineer (to chat_manager):

Sure! I will write a Python script to retrieve papers from the arXiv API based on the topic "Large Language Models (LLM) applications" and filter them to get the latest papers from the past week. The script will display the title, authors, abstract, and link for each paper.

```python
import requests
import datetime
from dateutil.parser import parse

def get_arxiv_papers(query, max_results=10):
    base_url = 'http://export.arxiv.org/api/query?'
    query = f'search_query=all:{query}&start=0&max_results={max_results}&sortBy=submittedDate&sortOrder=descending'
    
    response

No eligible speaker found. Terminating the conversation.


Message: [{"content": "Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.", "role": "user", "name": "Admin"}, {"content": "Sure! I will write a Python script to retrieve papers from the arXiv API based on the topic \"Large Language Models (LLM) applications\" and filter them to get the latest papers from the past week. The script will display the title, authors, abstract, and link for each paper.\n\n```python\nimport requests\nimport datetime\nfrom dateutil.parser import parse\n\ndef get_arxiv_papers(query, max_results=10):\n    base_url = 'http://export.arxiv.org/api/query?'\n    query = f'search_query=all:{query}&start=0&max_results={max_results}&sortBy=submittedDate&sortOrder=descending'\n    \n    response = requests.get(base_url + query)\n    \n    if response.status_code != 200:\n        print(\"Error fetching data from arXiv API\")\n        return\n    \n    feed = response.text\n    start_pos = 0\n    papers = []\n\n    while True:

ChatResult(chat_id=None, chat_history=[{'content': 'Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.', 'role': 'assistant', 'name': 'Admin'}, {'content': 'Sure! I will write a Python script to retrieve papers from the arXiv API based on the topic "Large Language Models (LLM) applications" and filter them to get the latest papers from the past week. The script will display the title, authors, abstract, and link for each paper.\n\n```python\nimport requests\nimport datetime\nfrom dateutil.parser import parse\n\ndef get_arxiv_papers(query, max_results=10):\n    base_url = \'http://export.arxiv.org/api/query?\'\n    query = f\'search_query=all:{query}&start=0&max_results={max_results}&sortBy=submittedDate&sortOrder=descending\'\n    \n    response = requests.get(base_url + query)\n    \n    if response.status_code != 200:\n        print("Error fetching data from arXiv API")\n        return\n    \n    feed = response.text\n    start_pos = 0\n

Save chat messages for resuming later on using the chat history

In [72]:
previous_state = manager.messages_to_string(manager.groupchat.messages)
print(previous_state)

[{"content": "Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.", "role": "user", "name": "Admin"}, {"content": "Sure! I will write a Python script to retrieve papers from the arXiv API based on the topic \"Large Language Models (LLM) applications\" and filter them to get the latest papers from the past week. The script will display the title, authors, abstract, and link for each paper.\n\n```python\nimport requests\nimport datetime\nfrom dateutil.parser import parse\n\ndef get_arxiv_papers(query, max_results=10):\n    base_url = 'http://export.arxiv.org/api/query?'\n    query = f'search_query=all:{query}&start=0&max_results={max_results}&sortBy=submittedDate&sortOrder=descending'\n    \n    response = requests.get(base_url + query)\n    \n    if response.status_code != 200:\n        print(\"Error fetching data from arXiv API\")\n        return\n    \n    feed = response.text\n    start_pos = 0\n    papers = []\n\n    while True:\n       

To resume, the agents, GroupChat, and GroupChatManager objects must exist and match the original group chat.

When preparing for resuming, the messages will be validated against the groupchat’s agents to make sure that the messages can be assigned to them. Messages will be allocated to the agents and then the last speaker and message will be returned for use in `initiate_chat`.

**Continuing a terminated conversation**

- If the previous group chat terminated and the resuming group chat has the same termination condition (such as if the message contains “TERMINATE”) then the conversation will terminate when resuming as the terminate check occurs with the message passed in to `initiate_chat`.

- If the termination condition is based on a string within the message, you can pass in that string in the `remove_termination_string` parameter of the `resume` function and it will be removed. If the termination condition is more complicated, you will need to adjust the messages accordingly before calling `resume`.

- The `resume` function will then check if the last message provided still meets the termination condition and warns you, if so.

Following is the code to resume a conversation from previous state

In [73]:
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config={"config_list": config_list}
)

In [74]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

In [75]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config= {"config_list": config_list},
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

In [76]:
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config={"config_list": config_list},
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)

In [77]:
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [78]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor],
    messages=[],
    max_round=10,
)

In [79]:
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config= { "config_list": config_list })

In [80]:
# Prepare the group chat for resuming
last_agent, last_message = manager.resume(messages=previous_state)

Prepared group chat with 4 messages, the last speaker is Scientist


In [81]:
# Resume the chat using the last agent and message
result = last_agent.initiate_chat(recipient=manager, message=last_message, clear_history=False)

Scientist (to chat_manager):

Here is a markdown table categorizing the papers based on their domains:

```markdown
| Domain                               | Title                                                                                      | Authors                                   | Summary                                                                                                                                                                                                                                                                        | Link                                     |
|--------------------------------------|--------------------------------------------------------------------------------------------|-------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**NOTE - Sometimes, the agents are not selected correctly. For best results, use your own `state_transition` speaker selection method as used previously.**

In [82]:
# Output the final chat history showing the original 4 messages and resumed messages
for i, message in enumerate(groupchat.messages):
    print(
        f"#{i + 1}, {message['name']}: {message['content'][:80]}".replace("\n", " "),
        f"{'...' if len(message['content']) > 80 else ''}".replace("\n", " "),
    )

#1, Admin: Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from d ...
#2, Engineer: Sure! I will write a Python script to retrieve papers from the arXiv API based o ...
#3, Executor: exitcode: 0 (execution succeeded) Code output:  Paper 1: Title: Video-Panda: Par ...
#4, Scientist: Here is a markdown table categorizing the papers based on their domains:  ```mar ...
#5, Admin: Approve 
#6, Planner: The plan is approved. The next steps are to share the summarized papers with the ...
#7, Admin: Approve 
#8, Admin: Approve 
#9, Admin: Approve 
#10, Admin:  
#11, Admin:  
#12, Admin:  
#13, Admin:  
